# Day 08. Exercise 02
# Multiclass classification. One-hot encoding. Random forest

## 0. Imports

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

## 1. Preprocessing

1. Read the file [`checker-submits.csv`](https://drive.google.com/file/d/14voc4fNJZiLEFaZyd8nEG-lQt5JjatYw/view?usp=sharing).
2. Create a dataframe `df` with the columns: `uid`, `labname`, `numTrials`, `hour`, `dayofweek` where `hour` is extracted from the `timestamp` as well as the `dayofweek` (`0` is Monday, `6` is Sunday). We will try to predict the day of the week having data about which user made a commit for which lab at which hour and which try it was.
3. Using `OneHotEncoder()` transform your categorical features, delete from the dataframe the initial columns.
4. Use `StandardScaler()` and scale your continuous features.
5. Save the dataframe as `dayofweek.csv`.
6. Before trying out different algorithms, find out the accuracy of the naive algorithms – the one that predicts everything as the most popular class.

Создадим датафрейм df со столбцами: uid, labname, numTrials, hour, dayofweek

Сначала посмотрим, что в исходном датафрейме

In [ ]:
df = pd.read_csv('../data/checker_submits.csv', parse_dates=['timestamp'])
df

Переделаем столбцы по заданию

In [ ]:
df['hour'] = df['timestamp'].dt.hour
df['dayofweek'] = df['timestamp'].dt.dayofweek
df = df.drop('timestamp', axis=1)
df

Мы попытаемся предсказать день недели, имея данные о том, какой пользователь сделал коммит, для какой лаборатории, в какое время и какая это была попытка.

Используя OneHotEncoder(), преобразуем свои категориальные функции, удалим из датафрейма исходные столбцы.

In [ ]:
df = pd.get_dummies(df, prefix=['uid', 'labname'], columns=['uid', 'labname'])
df

Используя StandardScaler(), масштабируем непрерывные функции.

In [ ]:
scaler = StandardScaler()
df[['numTrials', 'hour']] = scaler.fit_transform(df[['numTrials', 'hour']])
df

Сохраним датафрейм

In [ ]:
df.to_csv('../data/dayofweek.csv', index=False)

Прежде чем пробовать разные алгоритмы, узнаем точность наивного алгоритма — того, который все предсказывает как самый популярный класс.

In [ ]:
X = df.drop('dayofweek', axis=1)
y = df['dayofweek']
y_pred_naive = np.array([np.argmax(np.bincount(y))] * len(y))

Разработанный классификатор предсказывает к какому классу принадлежит объект на основании значений его признаков. В результате возможны четыре ситуации:

Объект имеет характеристику класса 1 и классификатор определяет характеристику как 1. Это означает, что классификатор сработал верно. Величина TP (True positive) содержит общее количество таких ответов.

Объект имеет характеристику класса 0, а классификатор определяет характеристику как 1. Это означает, что классификатор сработал не верно. Величина FP (False positive) содержит общее количество таких ответов.

Объект имеет характеристику класса 0 и классификатор определяет характеристику как 0. Это означает, что классификатор сработал верно. Величина TN (True negative) содержит общее количество таких ответов.

Объект имеет характеристику класса 1, а классификатор определяет характеристику как 0. Это означает, что классификатор сработал не верно. Величина FN (False negative) содержит общее количество таких ответов.

Accuracy определяет долю правильных ответов и считается следующим образом: 𝐴𝑐𝑐𝑢𝑟𝑎𝑐𝑦=𝑇𝑃+𝑇𝑁 / 𝑇𝑃+𝑇𝑁+𝐹𝑃+𝐹𝑁

In [ ]:
accuracy_score(y, y_pred_naive)

## 2. Algorithms

### a. Logreg

1. Train logistic regression, for the baseline model use `random_state=21`, `fit_intercept=False`. 
2. Calculate the accuracy.
3. Write a function that draws the plot (`barh`) taking coefficients of any trained models, names of the features and the number of `top-n` most important features to display.
4. Draw a plot (`barh`) for the baseline model with top-10 most important features (absolute value) for the trained model.
5. Remember that it is a multiclass classification and `coef_` returns a matrix, to calculate importance for a feature you need to sum all the individual feature importances for all the target values.

In [ ]:
lr = LogisticRegression(random_state=21, fit_intercept=False)
lr.fit(X, y)
y_pred = lr.predict(X)
accuracy_score(y, y_pred)

Построим график (barh) на основании рассчитанных коэффициентов регрессии для n наиболее значимых фичей

In [ ]:
def plot_features(coefs, features, n=10):
    fig, ax = plt.subplots(figsize=(15, 8)) # определяем параметры графика
    coefs /= coefs.sum() # коэффициенты берем с удельным весом
    indices = coefs.argsort()[::-1][:n] # индексами берем фичи (аргументы коэффициентов регрессии), отсортированные по возрастанию, поэтому срез с конца, последние n
    ax.barh(np.arange(n), coefs[indices], color='green')
    ax.set_yticks(np.arange(n))
    ax.set_yticklabels(features[indices])
    ax.invert_yaxis() # сделаем, чтобы более значимые фичи были сверху
    plt.show()

In [ ]:
plot_features(lr.coef_.mean(axis=0), X.columns)

### b. SVC

1. Train a `SVC` model, for the baseline model use parameters `kernel='linear'`, `probability=True`, `random_state=21`. 
2. Try different kernels, calculate the accuracies.
3. Draw a plot (`barh`) for the baseline model with top-10 most important features (absolute value) for the trained model for the linear kernel *

*By default SVC uses “one vs one” strategy of the classification, thus in `coef_` it returns a matrix. To calculate importance for a feature you need to use [OneVsRestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html) over the SVC and sum all the individual feature importances for all the target values.

SVS - метод опорных векторов.

http://datascientist.one/support-vector-machines/

Обучим модель SVC, используйя параметры ядра = 'линейный', вероятность = истина, random_state = 21

In [ ]:
svc = SVC(kernel='linear', probability=True, random_state=21)
svc.fit(X, y)
y_svc = svc.predict(X)

In [ ]:
accuracy_score(y, y_svc)

Нарисуйем график для базовой модели с 10 наиболее важными функциями (абсолютное значение веса) для обученной модели для линейного ядра

In [ ]:
plot_features(svc.coef_.mean(axis=0), X.columns)

Пробуем разные ядра, посчитаем точность.

In [ ]:
svc = SVC(probability=True, random_state=21)
param_grid = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}
gs = GridSearchCV(svc, param_grid, scoring='accuracy')
gs.fit(X, y)
gs.best_params_

Лучшим ядром оказалось poly

In [ ]:
y_pred = gs.predict(X)
accuracy_score(y, y_pred)

### c. Decision tree

1. Train a `DecisionTreeClassifier` using for the baseline model `max_depth=4`, `random_state=21`. 
2. Try different values of `max_depth`, calculate the accuracies.
3. Draw a plot (`barh`) for the baseline model with top-10 most important features (absolute value) for the trained model using the written function.

Обучим DecisionTreeClassifier, используя для базовой модели max_depth=4, random_state=21.

In [ ]:
dtc = DecisionTreeClassifier(max_depth=4, random_state=21)
dtc.fit(X, y)
y_dts = dtc.predict(X)
accuracy_score(y, y_dts)

Попробуем разные значения max_depth, рассчитаем точность.

In [ ]:
dtc = DecisionTreeClassifier(max_depth=6, random_state=21)
dtc.fit(X, y)
y_dts = dtc.predict(X)
accuracy_score(y, y_dts)

In [ ]:
dtc = DecisionTreeClassifier(max_depth=8, random_state=21)
dtc.fit(X, y)
y_dts = dtc.predict(X)
accuracy_score(y, y_dts)

In [ ]:
dtc = DecisionTreeClassifier(max_depth=10, random_state=21)
dtc.fit(X, y)
y_dts = dtc.predict(X)
accuracy_score(y, y_dts)

In [ ]:
dtc = DecisionTreeClassifier(max_depth=12, random_state=21)
dtc.fit(X, y)
y_dts = dtc.predict(X)
accuracy_score(y, y_dts)

In [ ]:
dtc = DecisionTreeClassifier(max_depth=28, random_state=21)
dtc.fit(X, y)
y_dts = dtc.predict(X)
accuracy_score(y, y_dts)

При увеличении глубины точность увеличивается. При глубине 28 достигается 1. Система становится переобученной.

Нарисуем график для базовой модели с 10 наиболее важными фичами (по абсолютному значению) для обученной модели, используя написанную функцию.

In [ ]:
dtc = DecisionTreeClassifier(random_state=21)
param_grid = {'max_depth': [3, 5, 7, 10, 20, 28]}
gs = GridSearchCV(dtc, param_grid, scoring='accuracy', n_jobs=-1)
gs.fit(X, y)

In [ ]:
gs.best_params_

In [ ]:
y_pred = gs.predict(X)
accuracy_score(y, y_pred)

In [ ]:
plot_features(gs.best_estimator_.feature_importances_, X.columns)

### d. Random forest

In real life forest is a set of trees. The same thing is with machine learning. Random forest is a set of individual decision trees (check the documentation for more details).

1. Train a `RandomForestClassifier` using for the baseline model parameters `n_estimators=100`, `max_depth = 25`, `random_state=21`. 
2. Try different values of `max_depth` and `n_estimators`, calculate the accuracies.
3. Draw a plot (`barh`) for the baseline model with top-10 most important features (absolute value) for the trained model using the written function.

Обучим RandomForestClassifier, используя параметры базовой модели n_estimators=100, max_depth = 25, random_state=21.

In [ ]:
rfm = RandomForestClassifier(n_estimators=100,
                             max_depth=25,
                             random_state=21)
rfm.fit(X, y) # обучение
y_rfm = rfm.predict(X) # предсказание

accuracy_score(y, y_rfm)

Попробуем разные значения max_depth и n_estimators, рассчитаем точность.

In [ ]:
rfm = RandomForestClassifier(n_estimators=10,
                             max_depth=25,
                             random_state=21)
rfm.fit(X, y) # обучение
y_rfm = rfm.predict(X) # предсказание

accuracy_score(y, y_rfm)

In [ ]:
rfm = RandomForestClassifier(n_estimators=100,
                             max_depth=20,
                             random_state=21)
rfm.fit(X, y) # обучение
y_rfm = rfm.predict(X) # предсказание

accuracy_score(y, y_rfm)

Нарисуем график для базовой модели с 10 наиболее важными функциями (по абсолютному значению) для обученной модели, используя написанную функцию.

In [ ]:
rfm = RandomForestClassifier(random_state=21)
param_grid = {'n_estimators': [100, 200, 300],
              'max_depth': [3, 5, 7, 10, 20, 25, 30]}
gs = GridSearchCV(rfm, param_grid, scoring='accuracy', n_jobs=-1)
gs.fit(X, y)

In [ ]:
gs.best_params_

In [ ]:
plot_features(gs.best_estimator_.feature_importances_, X.columns)